# Collaboration and Competition

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

In [ ]:
env = UnityEnvironment(file_name="./Tennis_Windows_x86_64/Tennis.exe")

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
print('The state for the second agent looks like:', states[1])

### Benchmark model

First a benchmark model is implemented. Therefore the multi agent problem is reformulated into a single DDPG problem. There is only one policy which receives both state vectors and outputs the actions of both agents. Also the critic receives both states and the actions of both actors.

In [ ]:
class environment_single_wrapper():
    def __init__(self, env):
        self.env = env
        self.vector_observations = self.env
        self.brain_names = self.env.brain_names
        
    def reset(self, train_mode):
        self.env_info = self.env.reset(train_mode)
        return self
    
    def __getitem__(self, brain_name):
        self.env_info = self.env_info[brain_name]
        self.vector_observations = self.env_info.vector_observations.reshape(1,-1)
        self.rewards = np.array([sum(self.env_info.rewards)]).reshape(1,-1)
        self.local_done = np.array([max(self.env_info.local_done)]).reshape(1,-1)
        return self
    
    def step(self, action):
        self.env_info = self.env.step(action)
        return self

In [ ]:
BUFFER_SIZE = 10000  # replay buffer size
BATCH_SIZE = 16        # minibatch size
GAMMA = 0.98            # discount factor
TAU = 5e-4              # for soft update of target parameters
LR_ACTOR = 1e-4       # learning rate of the actor 
LR_CRITIC = 1e-4       # learning rate of the critic

In [ ]:
from pi_network import PiNetwork
from q_network import QNetwork

actor = PiNetwork(input_size=state_size*2, output_size=action_size*2, layer_size1=128, layer_size2=64, lr=LR_ACTOR)
critic = QNetwork(input_size=state_size*2+action_size*2, layer_size1=128, layer_size2=128, lr=LR_CRITIC)

from agent import  DeterministicActorCriticAgent
from noise import OUNoise, GaussianNoise
from replay_buffer import ReplayBuffer
from trainer import Trainer


memory = ReplayBuffer(state_size, BUFFER_SIZE , BATCH_SIZE, 1337)
#noise = OUNoise(action_size*2, 1, 1337, sigma_decay=0.999, sigma=0.5, sigma_min=0.025)
noise = GaussianNoise(action_size*2, 1, 1337, sigma=1.0, sigma_decay=0.999, sigma_min=0.05)
ac =  DeterministicActorCriticAgent(gamma=GAMMA, tau=TAU, batch_size=BATCH_SIZE, update_every=1,
                                    actor=actor, critic=critic, memory=memory, noise=noise)

trainer = Trainer(environment_single_wrapper(env), ac)

scores = trainer.train(6000)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd


plt.figure(figsize=(15,8))
pd.Series(scores, name='DDPG').plot()
pd.Series(scores, name='Rolling mean DDPG (length: 100)').rolling(100).mean().plot()
plt.title('Score: Tennis environment')
plt.legend()

In [ ]:
import torch
actor = trainer.agent.actor_local#.state_dict()

#path = './coop_policy_network_tennis'
#actor = trainer.agent.actor_local.state_dict()
#torch.save(trainer.agent.actor_local.state_dict(), path)

#actor = PiNetwork(input_size=state_size*2, output_size=action_size*2, layer_size1=128, layer_size2=64, lr=LR_ACTOR)
#actor.load_state_dict(torch.load(path))
#actor.eval()

In [ ]:
env = environment_single_wrapper(env)

In [ ]:
mean_scores = []
for i in range(1, 100):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = []                  # initialize the score (for each agent)
    while True:
        #actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = actor(torch.from_numpy(states).float()).cpu().data.numpy()
        #actions = trainer.agent.act(states)
        #actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores.append(env_info.rewards)                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    
    max_score_of_episode = np.max(np.sum(np.array(scores).reshape(-1, rewards.shape[0]), axis=0))
    mean_scores.append(max_score_of_episode)
    print('Score (max over agents) from episode {}: {}'.format(i, max_score_of_episode))

print('Mean score over all episodes {}'.format(np.mean(mean_scores)))